In [1]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import numpy as np

Using TensorFlow backend.


In [0]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 20 # we iterate 20 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

In [0]:
num_train, height, width, depth = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
max_x_values = np.max(X_train)
X_train /= max_x_values # Normalise data to [0, 1] range
X_test /= max_x_values # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels


In [5]:
depth, height, width

(3, 32, 32)

In [0]:
inp = Input(shape=(height, width, depth))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Conv2D(conv_depth_1, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Conv2D(conv_depth_1, kernel_size, padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Conv2D(conv_depth_2, kernel_size, padding='same', activation='relu')(drop_1)
conv_4 = Conv2D(conv_depth_2, kernel_size, padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)


In [0]:
model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy


In [8]:
hist = model.fit(X_train, Y_train, # Train the model using the training set...
                 batch_size=batch_size, epochs=num_epochs,
                 verbose=2, validation_split=0.1) # ...holding out 10% of the data for validation

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
 - 26s - loss: 1.5107 - acc: 0.4457 - val_loss: 1.1034 - val_acc: 0.6186
Epoch 2/20
 - 24s - loss: 1.1124 - acc: 0.6005 - val_loss: 0.9123 - val_acc: 0.6842
Epoch 3/20
 - 24s - loss: 0.9503 - acc: 0.6616 - val_loss: 0.7941 - val_acc: 0.7270
Epoch 4/20
 - 24s - loss: 0.8603 - acc: 0.6970 - val_loss: 0.8011 - val_acc: 0.7246
Epoch 5/20
 - 24s - loss: 0.7935 - acc: 0.7184 - val_loss: 0.7318 - val_acc: 0.7472
Epoch 6/20
 - 24s - loss: 0.7411 - acc: 0.7389 - val_loss: 0.7205 - val_acc: 0.7562
Epoch 7/20
 - 24s - loss: 0.6982 - acc: 0.7536 - val_loss: 0.7082 - val_acc: 0.7600
Epoch 8/20
 - 24s - loss: 0.6634 - acc: 0.7662 - val_loss: 0.7014 - val_acc: 0.7580
Epoch 9/20
 - 24s - loss: 0.6274 - acc: 0.7777 - val_loss: 0.6532 - val_acc: 0.7788
Epoch 10/20
 - 24s - loss: 0.5976 - acc: 0.7892 - val_loss: 0.6769 - val_acc: 0.7710
Epoch 11/20
 - 24s - loss: 0.5730 - acc: 0.7976 - val_loss: 0.6284 - val_acc: 0.7902
Epoch 12/20
 - 24s - loss

In [9]:
model.evaluate(X_test, Y_test, verbose=0) # Evaluate the trained model on the test set!

[0.6881342557430268, 0.7842]

In [10]:
hist